In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy import *
import yfinance as yf

In [194]:
#creating the class stock
class Stock:
    def __init__(self, ticker) -> None:
        self.ticker = ticker
        self.historical_prices = yf.download(ticker) 
    def CurPrice(self, curDate):
        try:
            return self.historical_prices.loc[curDate]['Close']
        except KeyError:
            return None
    
    def NDayRet(self, N,curDate):
            curDate = pd.to_datetime(curDate)
            start_date_price = self.historical_prices.loc[curDate - pd.Timedelta(days=N)]['Close']
            end_date_price = self.CurPrice(curDate)
            return (end_date_price-start_date_price)/start_date_price
    def DailyRet(self, curDate):
        return self.NDayRet(1, curDate)
    def Last30daysPrice(self, curDate):
        array = []
        curDate = pd.to_datetime(curDate)
        for i in range(30):
            curDate = curDate - pd.Timedelta(days=i)
            array.append(self.CurPrice(curDate))
        return array

infosys = Stock('INFY.NS')
inf_data = infosys.historical_prices
# print(inf_data[1000:1005])

obj = Stock('INFY.NS')
print(obj.CurPrice('2024-01-04'))
print(obj.NDayRet(5, '2024-01-01'))
print(obj.Last30daysPrice('2011-11-02'))


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
1512.699951171875
-0.010050411744860814
[354.29998779296875, 354.57501220703125, None, None, None, 337.6937561035156, 334.91876220703125, 305.8687438964844, 305.51873779296875, None, 292.3062438964844, None, 299.9125061035156, 341.57501220703125, 344.0562438964844, 369.5562438964844, None, 350.3812561035156, None, 367.79376220703125, 409.39373779296875, 378.86248779296875, 392.73126220703125, None, 434.76873779296875, None, 371.3687438964844, 377.32501220703125, 376.7875061035156, 345.65625]


In [ ]:
inf_data.columns

all tickers of nifty50

In [2]:
nifty50_tickers = ['ACC.NS', 'ADANIPORTS.NS', 'AMBUJACEM.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 
                 'BANKBARODA.NS', 'BHARTIARTL.NS', 'BHEL.NS', 'BPCL.NS', 'CIPLA.NS', 
                 'COALINDIA.NS', 'DRREDDY.NS', 'GAIL.NS', 'GRASIM.NS', 'HCLTECH.NS', 
                 'HDFCBANK.NS',  'HINDALCO.NS', 'HINDUNILVR.NS', 'ICICIBANK.NS', 
                 'IDEA.NS', 'INDUSINDBK.NS', 'INFY.NS', 'ITC.NS', 'KOTAKBANK.NS', 
                 'LT.NS', 'LUPIN.NS', 'M&M.NS', 'MARUTI.NS', 'NTPC.NS', 'ONGC.NS', 
                 'POWERGRID.NS', 'PNB.NS', 'RELIANCE.NS', 'SBIN.NS', 'SUNPHARMA.NS', 
                 'TCS.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS', 'TECHM.NS', 'ULTRACEMCO.NS', 
                 'VEDL.NS', 'WIPRO.NS', 'ZEEL.NS', 'YESBANK.NS']

### Active stock selection strategy: 

In [6]:
#active stock selection strategy
# def stock_selection(end_month_date, N):
#     # selected_stocks = [ticker for ticker in nifty50_tickers if Stock(ticker).NDayRet(N, end_month_date) > 0]
#     selected_stocks = []
#     for ticker in nifty50_tickers:
#         stock = Stock(ticker)
#         if not stock.empty:

#             if stock.NDayRet(N, end_month_date) > 0:
#                 selected_stocks.append(ticker)
#     return selected_stocks
def stock_selection(tickers, historical_data, end_date, N):
    selected_stocks = []
    for ticker in tickers:
        stock_data = historical_data[ticker]
        if not stock_data.empty:
            N_day_return = stock_data['Adj Close'].pct_change(periods=N).loc[end_date]
            if N_day_return > 0:
                selected_stocks.append(ticker)
    return selected_stocks

In [ ]:
# stock_selection('2013-06-30')

### summarize the performance

In [ ]:
def calculate_performance_metrics(prices, t):
    daily_return = prices.pct_change().dropna()
    CAGR = ((prices.iloc[-1] / prices.iloc[0]) ** (1/t) -1) * 100   #Compound Annual Growth Rate
    Volatility = (std(daily_return) * sqrt(252)) * 100
    Sharpe_ratio = (sqrt(252) * mean(daily_return) / daily_return.std()) 
    return CAGR, Volatility, Sharpe_ratio

In [ ]:
# Download historical data for Infosys
infosys = yf.Ticker('INFY.NS')
hist = infosys.history()

# Calculate performance metrics
number_of_years = 5
cagr, volatility, sharpe_ratio = calculate_performance_metrics(hist['Close'], 2)

print(f"CAGR: {cagr * 100:.2f}%")
print(f"Volatility: {volatility * 100:.2f}%")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")


In [ ]:
nifty50_data

In [ ]:
# Download historical data for the Nifty 50 index
nifty50_data = yf.download('^NSEI', start='2019-01-01', end='2022-01-01')

# Calculate the number of years for the CAGR calculation
t_years = (nifty50_data.index[-1] - nifty50_data.index[0]).days / 365.25

# Calculate metrics for the Benchmark Allocation (Nifty 50 index)
benchmark_cagr, benchmark_volatility, benchmark_sharpe_ratio = calculate_performance_metrics(nifty50_data, t_years)

print(f"Benchmark Allocation CAGR: {benchmark_cagr:.2f}%")
print(f"Benchmark Allocation Volatility: {benchmark_volatility:.2f}%")
print(f"Benchmark Allocation Sharpe Ratio: {benchmark_sharpe_ratio:.2f}")


In [7]:
# Function to simulate the Sample Strategy
def simulate_sample_strategy(tickers, start_date, end_date, initial_equity, N):
    # Download historical data for all Nifty 50 stocks
    historical_data = {ticker: yf.download(ticker, start=start_date, end=end_date) for ticker in tickers}
    
    # Initialize portfolio value and equity curve
    portfolio_value = initial_equity
    equity_curve = pd.Series(index=pd.date_range(start=start_date, end=end_date), dtype=float)
    stock_values = {}  # Initialize stock_values outside the loop
    
    # Simulate the strategy over the given date range
    current_date = pd.to_datetime(start_date)
    while current_date <= pd.to_datetime(end_date):
        if current_date.month != (current_date + pd.Timedelta(days=1)).month:
            # End of the month: select stocks and reallocate portfolio
            selected_stocks = stock_selection(tickers, historical_data, current_date, N)
            if selected_stocks:
                # Assume equal allocation for simplicity
                allocation_per_stock = portfolio_value / len(selected_stocks)
                stock_values = {ticker: allocation_per_stock / historical_data[ticker]['Adj Close'].get(current_date, np.nan) for ticker in selected_stocks}
            else:
                stock_values = {}  # Reset stock_values if no stocks are selected
        # Update portfolio value
        portfolio_value = sum(historical_data[ticker]['Adj Close'].get(current_date, np.nan) * shares for ticker, shares in stock_values.items())
        equity_curve[current_date] = portfolio_value
        current_date += pd.Timedelta(days=1)
    
    # Calculate performance metrics
    sample_strategy_cagr, sample_strategy_volatility, sample_strategy_sharpe_ratio = calculate_performance_metrics(equity_curve.dropna(), t_years)
    
    return equity_curve.dropna(), sample_strategy_cagr, sample_strategy_volatility, sample_strategy_sharpe_ratio


# Simulate the Sample Strategy
equity_curve, sample_strategy_cagr, sample_strategy_volatility, sample_strategy_sharpe_ratio = simulate_sample_strategy(
    nifty50_tickers, '2019-01-01', '2022-01-01', 1000000, 30)

print(f"Sample Strategy CAGR: {sample_strategy_cagr:.2f}%")
print(f"Sample Strategy Volatility: {sample_strategy_volatility:.2f}%")
print(f"Sample Strategy Sharpe Ratio: {sample_strategy_sharpe_ratio:.2f}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

C:\Users\amana\AppData\Local\Temp\ipykernel_15200\1064553251.py:24: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  portfolio_value = sum(historical_data[ticker]['Adj Close'].get(current_date, np.nan) * shares for ticker, shares in stock_values.items())


KeyError: Timestamp('2019-03-31 00:00:00')

In [ ]:
historical_data = {}
def simulate_sample_strategy(nifty50_tickers, start_date, end_date, initial_equity, N):
    for ticker in nifty50_tickers:
            historical_data[ticker] = yf.download(ticker, start=start_date, end=end_date)

In [ ]:
data = {}
for ticker in nifty50_tickers:

    data[ticker] = yf.download(ticker, start='2017-01-01', end='2017-04-21')

In [ ]:
data

In [ ]:
data

In [ ]:
historical_data = {ticker: yf.download(ticker) for ticker in ticker}


In [ ]:
# Function to plot equity curves
def plot_equity_curves(nifty_equity_curve, sample_strategy_equity_curve):
    plt.figure(figsize=(14, 7))
    plt.plot(nifty_equity_curve, label='Nifty Index')
    plt.plot(sample_strategy_equity_curve, label='Sample Strategy')
    plt.title('Equity Curves Comparison')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value')
    plt.legend()
    plt.show()

# Main function to run the simulation and display results
def run_simulation(start_date, end_date, N, initial_equity):
    # Download Nifty 50 index data
    nifty50_data = yf.download('^NSEI', start=start_date, end=end_date)['Adj Close']
    
    # Calculate performance metrics for the Nifty Index
    nifty_cagr, nifty_volatility, nifty_sharpe_ratio = calculate_performance_metrics(nifty50_data, (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days / 365.25)
    
    # Simulate the Sample Strategy
    equity_curve, sample_strategy_cagr, sample_strategy_volatility, sample_strategy_sharpe_ratio = simulate_sample_strategy(
        nifty50_tickers, start_date, end_date, initial_equity, N)
    
    # Display the equity curves
    plot_equity_curves(nifty50_data, equity_curve)
    
    # Display the selected stocks for the Sample Strategy
    historical_data = {ticker: yf.download(ticker, start=start_date, end=end_date) for ticker in nifty50_tickers}
    last_month_end = pd.to_datetime(end_date) - pd.offsets.MonthEnd(1)
    selected_stocks = stock_selection(nifty50_tickers, historical_data, last_month_end, N)
    print(f"Selected stocks for the Sample Strategy as of {last_month_end.date()}: {selected_stocks}")
    
    # Display performance metrics
    print(f"Nifty Index CAGR: {nifty_cagr:.2f}%")
    print(f"Nifty Index Volatility: {nifty_volatility:.2f}%")
    print(f"Nifty Index Sharpe Ratio: {nifty_sharpe_ratio:.2f}")
    
    print(f"Sample Strategy CAGR: {sample_strategy_cagr:.2f}%")
    print(f"Sample Strategy Volatility: {sample_strategy_volatility:.2f}%")
    print(f"Sample Strategy Sharpe Ratio: {sample_strategy_sharpe_ratio:.2f}")

# Inputs
start_date = '2019-01-01'
end_date = '2022-01-01'
N = 30  # Number of days to measure performance for stock selection
initial_equity = 1000000  # Initial equity

# Run the simulation
run_simulation(start_date, end_date, N, initial_equity)


In [188]:
nifty50_data = yf.download('^NSEI', start='2019-01-01', end='2022-01-01')

[*********************100%%**********************]  1 of 1 completed


In [189]:
nifty50_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,10868.849609,10895.349609,10735.049805,10792.500000,10792.500000,309700
2019-01-03,10796.799805,10814.049805,10661.250000,10672.250000,10672.250000,286200
2019-01-04,10699.700195,10741.049805,10628.650391,10727.349609,10727.349609,296600
2019-01-07,10804.849609,10835.950195,10750.150391,10771.799805,10771.799805,269400
2019-01-08,10786.250000,10818.450195,10733.250000,10802.150391,10802.150391,277700
...,...,...,...,...,...,...
2021-12-27,16937.750000,17112.050781,16833.199219,17086.250000,17086.250000,144800
2021-12-28,17177.599609,17250.250000,17161.150391,17233.250000,17233.250000,176000
2021-12-29,17220.099609,17285.949219,17176.650391,17213.599609,17213.599609,161700
